# Final Report

In [ ]:
By: Alissa Stuart

## Load the Libraries

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests


!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 


import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



### Credentials for Foursquare Database

In [2]:
CLIENT_ID = 'RAGTNNCF4ZGYUGELKKUXFE3INSVV5VB3IFODBJTUWOOPZBWI' 
CLIENT_SECRET = '03JCWUSOLMU5PCWBCLYJYPLYSWVQMM31PWDDM314GKZBKRJ3' 
VERSION = '20180605' 
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RAGTNNCF4ZGYUGELKKUXFE3INSVV5VB3IFODBJTUWOOPZBWI
CLIENT_SECRET:03JCWUSOLMU5PCWBCLYJYPLYSWVQMM31PWDDM314GKZBKRJ3


### Function for gathering the location of venues from Foursquare

In [3]:
def getNearbyVenues(names, latitudes, longitudes, radius=4000):
    LIMIT=1000
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Gather Neighborhoods locations in Toronto from Wikipedia

In [4]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url, 'lxml')

table = str(soup.find("table", { "class" : "wikitable sortable" }))

df =pd.read_html(table)[0]
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df.drop(df.index[0], inplace=True)
df = df[df['Borough']!="Not assigned"]
df.Neighborhood[df.Neighborhood=="Not assigned"] = df.Borough[df.Neighborhood=="Not assigned"]
df=df.groupby(['PostalCode']).agg({'Borough': 'first','Neighborhood':', '.join})
df.reset_index(inplace=True)
df.shape

url="http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"
coordinates=pd.read_csv(url)
coordinates.columns = ['PostalCode', 'Latitude', 'Longitude']
df1 = pd.merge(df,coordinates, on="PostalCode")

df1 = df1[df1['Borough'].str.contains('Toronto')].reset_index(drop=True)
df1.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


### Gather all venues in Toronto from Foursquare - Use the locations of Neighborhoods as inputs

In [5]:
venues = getNearbyVenues(names=df1['Neighborhood'],
                        latitudes=df1['Latitude'],
                        longitudes=df1['Longitude']
                                  )
venues.columns =["Neighborhood","Neighborhood Latitude", "Neighborhood Longitude", "Venue", "Latitude", "Longitude", "Category" ]
venues.head(10)

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Latitude,Longitude,Category
0,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
1,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
2,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
3,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
4,The Beaches,43.676357,-79.293031,Kew Gardens,43.669038,-79.298538,Park
5,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
6,The Beaches,43.676357,-79.293031,Mastermind Toys,43.671453,-79.293971,Toy / Game Store
7,The Beaches,43.676357,-79.293031,Bagels On Fire,43.672864,-79.286784,Bagel Shop
8,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
9,The Beaches,43.676357,-79.293031,I'm On The Beach,43.670364,-79.284696,Beach


### Find the coffee shop, restaurants, bars and hotels in venues and create a new dataframe for each of them

In [6]:
coffee_shops = venues[venues['Category']=='Coffee Shop']
restaurants = venues[venues['Category'].str.contains('Restaurant') ==1]
bars = venues[(venues['Category'].str.contains('Bar') ==1) & (venues['Category'].str.contains('Barbershop')==0)]
hotels = venues[(venues['Category'].str.contains('Hotel') ==1) ]

In [7]:
(coffee_shops.head(10))

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Latitude,Longitude,Category
12,The Beaches,43.676357,-79.293031,The Remarkable Bean,43.672801,-79.287038,Coffee Shop
13,The Beaches,43.676357,-79.293031,Buds Coffee Bar,43.669375,-79.303218,Coffee Shop
49,The Beaches,43.676357,-79.293031,Press Books Coffee Vinyl,43.687672,-79.304457,Coffee Shop
76,The Beaches,43.676357,-79.293031,Pomarosa Coffee & Kitchen,43.683201,-79.325849,Coffee Shop
79,The Beaches,43.676357,-79.293031,Starbucks,43.682379,-79.327249,Coffee Shop
123,"The Danforth West, Riverdale",43.679557,-79.352188,Hailed Coffee,43.666900,-79.345432,Coffee Shop
132,"The Danforth West, Riverdale",43.679557,-79.352188,Merchants of Green Coffee,43.659916,-79.353963,Coffee Shop
147,"The Danforth West, Riverdale",43.679557,-79.352188,Te Aro,43.661373,-79.338577,Coffee Shop
149,"The Danforth West, Riverdale",43.679557,-79.352188,Dark Horse Espresso Bar,43.658498,-79.352356,Coffee Shop
172,"The Danforth West, Riverdale",43.679557,-79.352188,Rooster Coffee House,43.669654,-79.379871,Coffee Shop


In [8]:
(restaurants.head(10))

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Latitude,Longitude,Category
2,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
14,The Beaches,43.676357,-79.293031,Delina Restaurant,43.668867,-79.305404,Middle Eastern Restaurant
18,The Beaches,43.676357,-79.293031,Veloute Bistro,43.672267,-79.289584,French Restaurant
24,The Beaches,43.676357,-79.293031,Xola,43.672603,-79.288080,Mexican Restaurant
28,The Beaches,43.676357,-79.293031,Budapest Restaurant,43.680946,-79.310110,Hungarian Restaurant
30,The Beaches,43.676357,-79.293031,Isabella’s Boutique Restaurant,43.673767,-79.282703,Japanese Restaurant
34,The Beaches,43.676357,-79.293031,Lake Inez,43.672520,-79.320712,Asian Restaurant
38,The Beaches,43.676357,-79.293031,Green Eggplant,43.670517,-79.298660,Mediterranean Restaurant
39,The Beaches,43.676357,-79.293031,Udupi Palace,43.672480,-79.321275,Indian Restaurant
47,The Beaches,43.676357,-79.293031,Jatujak,43.688421,-79.270073,Thai Restaurant


In [9]:
(bars.head(10))

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Latitude,Longitude,Category
53,The Beaches,43.676357,-79.293031,Relish Bar & Grill,43.686280,-79.310980,Bar
77,The Beaches,43.676357,-79.293031,The Shore Leave,43.684200,-79.319413,Cocktail Bar
83,The Beaches,43.676357,-79.293031,Hitch Bar,43.663250,-79.330649,Bar
85,The Beaches,43.676357,-79.293031,Pinkerton Snack Bar,43.668900,-79.337309,Cocktail Bar
120,"The Danforth West, Riverdale",43.679557,-79.352188,The Only Cafe,43.680409,-79.337898,Beer Bar
159,"The Danforth West, Riverdale",43.679557,-79.352188,The Comrade,43.659346,-79.347932,Bar
161,"The Danforth West, Riverdale",43.679557,-79.352188,Rooftop At Broadview Hotel,43.659109,-79.350074,Hotel Bar
180,"The Danforth West, Riverdale",43.679557,-79.352188,Hitch Bar,43.663250,-79.330649,Bar
183,"The Danforth West, Riverdale",43.679557,-79.352188,FUEL+,43.664399,-79.380427,Juice Bar
188,"The Danforth West, Riverdale",43.679557,-79.352188,Greenhouse Juice Co,43.679101,-79.390686,Juice Bar


In [10]:
(hotels.head(10))

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Latitude,Longitude,Category
138,"The Danforth West, Riverdale",43.679557,-79.352188,The Broadview Hotel,43.659060,-79.350030,Hotel
161,"The Danforth West, Riverdale",43.679557,-79.352188,Rooftop At Broadview Hotel,43.659109,-79.350074,Hotel Bar
292,"The Beaches West, India Bazaar",43.668999,-79.315572,The Broadview Hotel,43.659060,-79.350030,Hotel
306,Studio District,43.659526,-79.340923,The Broadview Hotel,43.659060,-79.350030,Hotel
320,Studio District,43.659526,-79.340923,Rooftop At Broadview Hotel,43.659109,-79.350074,Hotel Bar
395,Studio District,43.659526,-79.340923,The Grand Hotel & Suites Toronto,43.656449,-79.374110,Hotel
826,"Moore Park, Summerhill East",43.689574,-79.383160,Four Seasons Hotel Toronto,43.671796,-79.389457,Hotel
921,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,Four Seasons Hotel Toronto,43.671796,-79.389457,Hotel
1011,Rosedale,43.679563,-79.377529,Four Seasons Hotel Toronto,43.671796,-79.389457,Hotel
1063,Rosedale,43.679563,-79.377529,The Grand Hotel & Suites Toronto,43.656449,-79.374110,Hotel


In [11]:
print(coffee_shops.shape)
print(restaurants.shape)
print(bars.shape)
print(hotels.shape)

(249, 7)
(961, 7)
(172, 7)
(81, 7)


### Markk the venues of interest in the map of Toronto

In [12]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, neighborhood in zip(restaurants['Latitude'], restaurants['Longitude'], restaurants['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
for lat, lng, neighborhood in zip(coffee_shops['Latitude'], coffee_shops['Longitude'], coffee_shops['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
for lat, lng, neighborhood in zip(bars['Latitude'], bars['Longitude'], bars['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='black',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
for lat, lng, neighborhood in zip(hotels['Latitude'], hotels['Longitude'], hotels['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

C:\Users\Alissa\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


### Concatanate the locations of the venues in a new dataframe - Find the central location and the standard deviation in location (x,y)

In [13]:
clients = pd.concat([coffee_shops,bars, restaurants, hotels])
data =clients[['Latitude', 'Longitude']].values
x0 =np.mean(data[:,0])
y0=np.mean(data[:,1])

### Perform a kmeans clustering to define the 5 clusters of venues (use the location features as inputs)

In [14]:
# set number of clusters
kclusters = 5
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(data)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

clients['Labels'] = kmeans.labels_

### Mark the venues again on the map of Toronto - Use different colors to indicate the clusters

In [15]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat, lng, cluster in zip(clients['Latitude'], clients['Longitude'], clients['Labels']):
    label = cluster
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

### Function returning the mean radius of each cluster

In [16]:
def mean_radius(cl,loc):
    num=0
    sum_r = 0
    for x,y in zip(cl['Latitude'],cl['Longitude']):
        r= np.sqrt(((x-loc[0])*110)**2+((y-loc[1])*110*np.cos(x))**2)
        
        sum_r = sum_r+r
        num=num+1
    mean_r = sum_r/num
   
    return mean_r,num

### Find the number of venues and the mean radius before clustering

In [17]:
cl=clients
loc=[x0,y0]
r,num_all=mean_radius(cl,loc)
print(loc,r, num_all)

[43.664529603253875, -79.3908134769568] 3.583506388745761 1463


### Find the number of venues and mean radius for each of the five clusters

In [18]:
locations =[]
radii=[]
num_per_cluster=[]
for i in range(kclusters):
    cl=clients[clients['Labels']==i]
    loc=kmeans.cluster_centers_[i]
    r,num=mean_radius(cl,loc)
    locations.append(loc)
    radii.append(r)
    num_per_cluster.append(num)
locations =np.array(locations)
raddi=np.array(radii)

### Calculate the density of Each Cluster

In [19]:
num_per_cluster

[367, 502, 191, 227, 176]

In [20]:
locations

array([[ 43.65181561, -79.40557884],
       [ 43.65460821, -79.37823502],
       [ 43.67045797, -79.33404346],
       [ 43.706888  , -79.39771575],
       [ 43.65827334, -79.44860756]])

In [21]:
3*np.array(radii)

array([3.77330428, 3.57517233, 5.42987478, 4.26871508, 6.18847369])

In [22]:
area = (np.pi*(3*np.array(radii))**2)
area

array([ 44.729447  ,  40.1553887 ,  92.6252729 ,  57.2458766 ,
       120.31422301])

In [23]:
(np.array(num_per_cluster))/area

array([ 8.2048857 , 12.50143545,  2.0620722 ,  3.9653511 ,  1.46283619])

### Mark each cluster on the map -Use variable radius for each cluster to indicate its radius

In [24]:
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, r, lbl in zip(locations[:,0], locations[:,1], radii, range(kclusters)):
    label = lbl
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=r*5000,
        popup=label,
        color=rainbow[lbl-1],
        fill=True,
        fill_color=rainbow[lbl-1],
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
    print(lat,lng,r)
map_toronto

43.65181561318263 -79.4055788364568 1.2577680930152335
43.65460821203199 -79.37823501566174 1.1917241108576107
43.67045797362043 -79.33404346119327 1.8099582584540885
43.70688800475742 -79.39771574613235 1.4229050278306474
43.65827333545963 -79.44860755957949 2.0628245638858997
